# 📈 Transaction Data Cleaning & Preprocessing Project



## 1. Introduction



This notebook documents the data cleaning and preprocessing steps performed on a raw transactional dataset. The goal is to transform the messy, incomplete, and incorrectly formatted data into a clean, structured, and analysis-ready format.



The data includes transaction details such as Item, Quantity, Price Per Unit, Total Spent, Payment Method, Location, and Transaction Date, which require significant imputation and correction.

In [ ]:
import pandas as pd
import numpy as np


## 2. Data Loading and Initial Inspection



First, the dataset is loaded into a pandas DataFrame. The initial inspection (df.head(), df.shape, and df.dtypes) reveals the size of the dataset (10,000 rows, 8 columns) and the critical observation that **all columns are of the generic object dtype**, indicating the presence of non-numeric and mixed-type data that needs cleaning.

In [2]:
file_path = r"C:\Users\tajes\Desktop\Portfolio one raw data.xlsx"
df = pd.read_excel(file_path)
df.head()

,Transaction ID,Item,Quantity,Price Per Unit,Total Spent,Payment Method,Location,Transaction Date
0,TXN_1961373,Coffee,2,2,4,Credit Card,Takeaway,2023-09-08 00:00:00
1,TXN_4977031,Cake,4,3,12,Cash,In-store,2023-05-16 00:00:00
2,TXN_4271903,Cookie,4,1,ERROR,Credit Card,In-store,2023-07-19 00:00:00
3,TXN_7034554,Salad,2,5,10,UNKNOWN,UNKNOWN,2023-04-27 00:00:00
4,TXN_3160411,Coffee,2,2,4,Digital Wallet,In-store,2023-06-11 00:00:00


In [3]:
df.shape

(10000, 8)

In [4]:
df.dtypes


Transaction ID      object
Item                object
Quantity            object
Price Per Unit      object
Total Spent         object
Payment Method      object
Location            object
Transaction Date    object
dtype: object

## 3. Data Cleaning: Standardization and Typing



### 3.1 Cleaning 'Item'



The unique values show inconsistent labels for missing data ('UNKNOWN', 'ERROR', and NaN). These are standardized to 'unknown' for unified handling in later steps.

In [5]:
df.Item.unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [6]:
df['Item'] = df['Item'].replace(['UNKNOWN', 'ERROR', np.nan], 'unknown')

In [7]:
df.Item.unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'unknown',
       'Sandwich', 'Juice', 'Tea'], dtype=object)

### 3.2 Cleaning 'Quantity' and 'Price Per Unit'



These columns contain non-numeric string values ('ERROR', 'UNKNOWN') and missing values (NaN).



1. Non-numeric/missing values are replaced with NaN using pd.to_numeric with errors='coerce'.


2. The NaN values are then imputed with a placeholder value of **-1** to signify a missing/invalid numerical entry before converting the column to an integer (int) or float (float) dtype.

In [8]:
df['Quantity'].unique()

array([2, 4, 5, 3, 1, 'ERROR', 'UNKNOWN', nan], dtype=object)

 <br> 3.2 Cleaning 'Quantity' and 'Price Per Unit'<br><br>These columns contain non-numeric string values (`'ERROR'`, `'UNKNOWN'`) and missing values (`NaN`).<br><br>1. Non-numeric/missing values are replaced with `NaN` using `pd.to_numeric` with `errors='coerce'`.<br>2. The `NaN` values are then imputed with a placeholder value of `-1` (an out-of-range value) to signify a missing/invalid numerical entry before converting the column to an integer (`int`) or float (`float`) dtype.<br

In [9]:
df['Quantity'] = pd.to_numeric(
    df['Quantity'].where(~df['Quantity'].isin(['ERROR', 'UNKNOWN']), np.nan),
    errors='coerce'
).fillna(-1).astype(int)


In [10]:

df["Price Per Unit"].unique()


array([2, 3, 1, 5, 4, 1.5, nan, 'ERROR', 'UNKNOWN'], dtype=object)

In [11]:
df['Price Per Unit'] = pd.to_numeric(
    df['Price Per Unit'].where(~df['Price Per Unit'].isin(['ERROR', 'UNKNOWN']), np.nan),
    errors='coerce'
).fillna(-1).astype(float)

In [12]:
df['Total Spent'].unique()

array([4, 12, 'ERROR', 10, 20, 9, 16, 15, 25, 8, 5, 3, 6, nan, 'UNKNOWN',
       2, 1, 7.5, 4.5, 1.5], dtype=object)

In [13]:
 df['Total Spent'] = np.where(df['Total Spent'].isin(['ERROR', 'UNKNOWN']) | df['Total Spent'].isna(), -1, df['Total Spent'])



In [14]:
df['Payment Method'].unique()

array(['Credit Card', 'Cash', 'UNKNOWN', 'Digital Wallet', 'ERROR', nan],
      dtype=object)

In [15]:
 df['Payment Method'] = np.where(df['Payment Method'].isin(['ERROR', 'UNKNOWN']) | df['Payment Method'].isna(), "Unknown", df['Payment Method'])
 df['Payment Method'].unique()

array(['Credit Card', 'Cash', 'Unknown', 'Digital Wallet'], dtype=object)

In [16]:
df.Location.unique()

array(['Takeaway', 'In-store', 'UNKNOWN', nan, 'ERROR'], dtype=object)

In [17]:
 df['Location'] = np.where(df['Location'].isin(['ERROR', 'UNKNOWN']) | df['Location'].isna(), "Unknown", df['Location'])
 df['Location'].unique()

array(['Takeaway', 'In-store', 'Unknown'], dtype=object)

In [18]:
df['Transaction Date'] = np.where(df['Transaction Date'].isin(['ERROR', 'UNKNOWN']) | df['Transaction Date'].isna(), "Unknown", df['Transaction Date'])

In [19]:
df['Transaction Date'].isnull().sum()

np.int64(0)

In [20]:
df.groupby("Item")["Price Per Unit"].value_counts().reset_index(name="count")

,Item,Price Per Unit,count
0,Cake,3.0,1085
1,Cake,-1.0,54
2,Coffee,2.0,1108
3,Coffee,-1.0,57
4,Cookie,1.0,1026
5,Cookie,-1.0,66
6,Juice,3.0,1110
7,Juice,-1.0,61
8,Salad,5.0,1082
9,Salad,-1.0,66


In [21]:
correct_prices = (
    df[df['Price Per Unit'] > 0]
    .groupby('Item')['Price Per Unit']
    .agg(lambda x: x.mode().iloc[0])
)

df['Price Per Unit'] = df['Price Per Unit'].mask(
    (df['Price Per Unit'].isna()) | (df['Price Per Unit'] == -1),
    df['Item'].map(correct_prices)
)


In [22]:

df.groupby("Item")["Price Per Unit"].value_counts().reset_index(name="count")

,Item,Price Per Unit,count
0,Cake,3.0,1139
1,Coffee,2.0,1165
2,Cookie,1.0,1092
3,Juice,3.0,1171
4,Salad,5.0,1148
5,Sandwich,4.0,1131
6,Smoothie,4.0,1096
7,Tea,1.5,1089
8,unknown,3.0,288
9,unknown,4.0,213


In [23]:
price_to_item = (
    df[df['Item'] != 'unknown']
    .groupby(['Price Per Unit', 'Item'])
    .size()
    .reset_index(name='count')
    .sort_values(['Price Per Unit', 'count'], ascending=[True, False])
    .drop_duplicates(subset='Price Per Unit', keep='first')
    .set_index('Price Per Unit')['Item']
    .to_dict()
)

df['Item'] = df['Item'].mask(
    df['Item'] == 'unknown',
    df['Price Per Unit'].map(price_to_item)
)


In [24]:
df.loc[(df['Total Spent'] == -1) & (df['Quantity'] > 0), 'Total Spent'] = \
    df['Quantity'] * df['Price Per Unit']

df.loc[(df['Quantity'] == -1) & (df['Total Spent'] > 0), 'Quantity'] = (
    (df['Total Spent'] / df['Price Per Unit']).round().astype(int)
)




In [25]:
invalid_totals = df[(df['Quantity'] == -1) | (df['Total Spent'] == -1)].copy()
invalid_totals['Expected_Total_Spent'] = np.where(
    invalid_totals['Quantity'] > 0,
    invalid_totals['Quantity'] * invalid_totals['Price Per Unit'],
    np.nan
)
invalid_totals['Expected_Quantity'] = np.where(
    invalid_totals['Total Spent'] > 0,
    invalid_totals['Total Spent'] / invalid_totals['Price Per Unit'],
    np.nan
)

invalid_totals['Check_Total'] = invalid_totals['Total Spent'] == invalid_totals['Expected_Total_Spent']
still_invalid = invalid_totals[~invalid_totals['Check_Total']]
print(still_invalid[['Item', 'Price Per Unit', 'Quantity', 'Total Spent',
                     'Expected_Quantity', 'Expected_Total_Spent']])



          Item  Price Per Unit  Quantity Total Spent Expected_Quantity  \
236      Salad             5.0        -1          -1               NaN   
278      Juice             3.0        -1          -1               NaN   
641      Juice             3.0        -1          -1               NaN   
738   Sandwich             4.0        -1          -1               NaN   
2796      Cake             3.0        -1          -1               NaN   
3203  Smoothie             4.0        -1          -1               NaN   
3224    Coffee             2.0        -1          -1               NaN   
3401       Tea             1.5        -1          -1               NaN   
4257    Coffee             2.0        -1          -1               NaN   
5841    Cookie             1.0        -1          -1               NaN   
7029    Coffee             2.0        -1          -1               NaN   
7297  Smoothie             4.0        -1          -1               NaN   
8021     Salad             5.0        

In [26]:
fully_unknown = df[(df['Quantity'] == -1) & (df['Total Spent'] == -1)]
print("Fully unknown rows:", len(fully_unknown))
partially_invalid = df[((df['Quantity'] != -1) | (df['Total Spent'] != -1)) &
                       (df['Total Spent'] != df['Quantity'] * df['Price Per Unit'])]
print("Partially invalid rows:", len(partially_invalid))
print("Fully unknown:")
print(fully_unknown[['Item', 'Price Per Unit', 'Quantity', 'Total Spent']])
print("\nPartially invalid:")
print(partially_invalid[['Item', 'Price Per Unit', 'Quantity', 'Total Spent']])


Fully unknown rows: 20
Partially invalid rows: 37
Fully unknown:
          Item  Price Per Unit  Quantity Total Spent
236      Salad             5.0        -1          -1
278      Juice             3.0        -1          -1
641      Juice             3.0        -1          -1
738   Sandwich             4.0        -1          -1
2796      Cake             3.0        -1          -1
3203  Smoothie             4.0        -1          -1
3224    Coffee             2.0        -1          -1
3401       Tea             1.5        -1          -1
4257    Coffee             2.0        -1          -1
5841    Cookie             1.0        -1          -1
7029    Coffee             2.0        -1          -1
7297  Smoothie             4.0        -1          -1
8021     Salad             5.0        -1          -1
8443  Sandwich             4.0        -1          -1
8465    Coffee             2.0        -1          -1
8479  Sandwich             4.0        -1          -1
8574     Juice             3.0    

In [27]:
df_clean = df[~((df['Quantity'] == -1) & (df['Total Spent'] == -1))]
df_clean = df_clean[df_clean['Total Spent'] == df_clean['Quantity'] * df_clean['Price Per Unit']]
df_clean = df_clean.reset_index(drop=True)
print(df_clean[['Item', 'Price Per Unit', 'Quantity', 'Total Spent']])
print("Remaining rows:", len(df_clean))


          Item  Price Per Unit  Quantity Total Spent
0       Coffee             2.0         2           4
1         Cake             3.0         4          12
2       Cookie             1.0         4         4.0
3        Salad             5.0         2          10
4       Coffee             2.0         2           4
...        ...             ...       ...         ...
9938  Sandwich             4.0         4          16
9939    Coffee             2.0         2           4
9940    Coffee             2.0         4           8
9941    Cookie             1.0         3           3
9942  Sandwich             4.0         3          12

[9943 rows x 4 columns]
Remaining rows: 9943


In [28]:
cols_to_check = ['Payment Method', 'Location', 'Transaction Date']
unknown_counts = {col: df_clean[col].isin(['Unknown', None, '']).sum() for col in cols_to_check}

for col, count in unknown_counts.items():
    print(f"{col}: {count} unknowns")



Payment Method: 3155 unknowns
Location: 3936 unknowns
Transaction Date: 459 unknowns


In [29]:
columns_to_keep = ['Transaction ID', 'Item', 'Quantity', 'Price Per Unit', 'Total Spent',
                   'Payment Method', 'Location', 'Transaction Date']
df_final = df_clean[columns_to_keep].copy()

df_final.to_csv('cleaned_data.csv', index=False)
print("Cleaned dataset saved! Total rows:", len(df_final))


Cleaned dataset saved! Total rows: 9943
